In [11]:
import numpy as np

In [3]:
import gradio as gr
import numpy as np
from PIL import Image
import tensorflow as tf
import os

# Initialize model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

model = create_model()  # Replace with your trained model

def process_gradio_image(img):
    """Handle both Sketchpad (numpy array) and Image (dict) outputs"""
    if isinstance(img, dict):
        img = img['image']  # Extract from dictionary if needed
    # Convert to grayscale if RGB/RGBA
    if len(img.shape) == 3:
        if img.shape[2] == 4:  # RGBA
            img = img[:, :, :3]  # Remove alpha
        img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])  # RGB to grayscale
        print("sadhsahdsahdahdjdsadsadsadsadsada")
    return img

def predict_digit(img):
    img = process_gradio_image(img)
    pred = model.predict(img)
    return {"Prediction": str(np.argmax(pred)),
           "Confidence": float(np.max(pred))}

def save_digit(img):
    img = process_gradio_image(img).reshape(28, 28)
    img = (img * 255).astype(np.uint8)
    os.makedirs("digits", exist_ok=True)
    img_path = f"digits/digit_{len(os.listdir('digits'))}.png"
    Image.fromarray(img).save(img_path)
    return f"Saved to {img_path}"


C:\Users\Lenovo\PycharmProjects\ML_Introduction\.venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [34]:
from PIL import Image
# convert to matrix form
matrix = np.array(Image.open("image7.jpg").convert('L'))
from helper import resize
# change background from white to black (invert)
inverted_image = matrix

# Resize if needed (ensure it's 28x28)
resized_image = resize(np.array(inverted_image))

# Flatten and reshape
x_input_flat = resized_image.reshape(1, -1)
X = correct_shape_array = x_input_flat[np.newaxis, :, :]  # Shape: (1, 28, 28)



# Predict
pred = model.predict(X)
print(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


In [5]:
import tkinter as tk
from tkinter import Canvas, Button, Label, filedialog
from PIL import Image, ImageDraw, ImageTk
import numpy as np
import tensorflow as tf

class DigitClassifier:
    def __init__(self, root):
        self.root = root
        self.root.title("MNIST Digit Classifier")

        # Initialize model (replace with your actual model loading)
        self.model = model

        # Drawing canvas
        self.canvas = Canvas(root, width=280, height=280, bg='white')
        self.canvas.pack()
        self.canvas.bind("<B1-Motion>", self.draw)

        # Image drawing setup
        self.image = Image.new("L", (280, 280), 0)
        self.draw_ref = ImageDraw.Draw(self.image)
        self.last_x, self.last_y = None, None

        # Buttons
        self.btn_frame = tk.Frame(root)
        self.btn_frame.pack()

        self.predict_btn = Button(self.btn_frame, text="Predict", command=self.predict)
        self.predict_btn.pack(side=tk.LEFT, padx=5)

        self.clear_btn = Button(self.btn_frame, text="Clear", command=self.clear_canvas)
        self.clear_btn.pack(side=tk.LEFT, padx=5)

        self.upload_btn = Button(self.btn_frame, text="Upload", command=self.upload_image)
        self.upload_btn.pack(side=tk.LEFT, padx=5)

        # Prediction label
        self.prediction = Label(root, text="Prediction: ", font=('Arial', 18))
        self.prediction.pack()

        # Keep reference to image
        self.tk_img = None



    def draw(self, event):
        """Draw on canvas"""
        x, y = event.x, event.y
        if self.last_x and self.last_y:
            self.canvas.create_line(self.last_x, self.last_y, x, y, width=10, capstyle=tk.ROUND, smooth=tk.TRUE)
            self.draw_ref.line([self.last_x, self.last_y, x, y], fill=255, width=10)
        self.last_x = x
        self.last_y = y

    def preprocess_image(self):
        """Convert drawing to MNIST format"""
        img = self.image.resize((28, 28))
        img_array = np.array(img) / 255.0
        return np.expand_dims(img_array, axis=0)  # Add batch dimension

    def predict(self):
        """Run prediction on drawn image"""
        try:
            img_array = self.preprocess_image()
            predictions = self.model.predict(img_array)
            digit = np.argmax(predictions)
            confidence = np.max(predictions)
            self.prediction.config(text=f"Prediction: {digit} (Confidence: {confidence:.2%})")
        except Exception as e:
            self.prediction.config(text=f"Error: {str(e)}")

    def clear_canvas(self):
        """Clear the drawing canvas"""
        self.canvas.delete("all")
        self.image = Image.new("L", (280, 280), 0)
        self.draw_ref = ImageDraw.Draw(self.image)
        self.last_x, self.last_y = None, None
        self.prediction.config(text="Prediction: ")

    def upload_image(self):
        """Upload an image file"""
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            try:
                img = Image.open(file_path).convert('L')
                img = img.resize((280, 280))
                self.image = img
                self.draw_ref = ImageDraw.Draw(self.image)
                self.canvas.delete("all")
                self.tk_img = ImageTk.PhotoImage(img)
                self.canvas.create_image(0, 0, image=self.tk_img, anchor='nw')
            except Exception as e:
                self.prediction.config(text=f"Upload error: {str(e)}")

if __name__ == "__main__":
    root = tk.Tk()
    app = DigitClassifier(root)
    root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
